In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pr as erk

# Load the industry returns and the total market index we previously created
ind_return = erk.get_ind_returns()

In [2]:
import pandas as pd
import numpy as np

risky_r = ind_return[["TUR"]]
# Assume the safe asset is paying 3% per year
safe_r = ind_return[["TECH"]]
start = 1000 # start at $1000
floor = 0.80 # set the floor to 80 percent of the starting value

In [5]:
print(safe_r)

           TECH
2015-02  0.0959
2015-03 -0.0436
2015-04  0.0174
2015-05  0.0339
2015-06 -0.0478
...         ...
2023-08 -0.0344
2023-09 -0.0731
2023-10  0.0133
2023-11  0.1301
2023-12  0.0004

[107 rows x 1 columns]


In [6]:
print(risky_r)

            TUR
2015-02  0.0654
2015-03 -0.0025
2015-04 -0.0568
2015-05  0.0109
2015-06 -0.0353
...         ...
2023-08  0.0296
2023-09 -0.0611
2023-10 -0.0569
2023-11  0.1102
2023-12  0.0855

[107 rows x 1 columns]


In [3]:
# set up the CPPI parameters
dates = risky_r.index
n_steps = len(dates)
account_value = start
floor_value = start*floor
m = 4
## set up some DataFrames for saving intermediate values
account_history = pd.DataFrame().reindex_like(risky_r)
risky_w_history = pd.DataFrame().reindex_like(risky_r)
cushion_history = pd.DataFrame().reindex_like(risky_r)

In [4]:
for step in range(n_steps): # the budy of this loop is going to be executed for first with step, first with step equal to zero then with step equal to one 
    cushion = (account_value - floor_value)/account_value
    risky_w = m*cushion # how much momey do I give to the risk yasset
    risky_w = np.minimum(risky_w, 1)
    risky_w = np.maximum(risky_w, 0)
    safe_w = 1-risky_w # # how much momey do I give to the safe asset
    risky_alloc = account_value*risky_w # how many dolars hace you allocated to the risky asset
    safe_alloc = account_value*safe_w # how many dolars hace you allocated to the safe asset
    # recompute the new account value at the end of this step
    account_value = risky_alloc*(1+risky_r.iloc[step]) + safe_alloc*(1+safe_r.iloc[step]) # Grow the risky allocation by that return and grow the safe allocation by that return
    # save the histories for analysis and plotting
    cushion_history.iloc[step] = cushion
    risky_w_history.iloc[step] = risky_w
    account_history.iloc[step] = account_value
    risky_wealth = start*(1+risky_r).cumprod() # To compare, if I just stock my money in he risky asset, start with a 1000$ and then compounding it over time

ValueError: could not broadcast input array from shape (2,) into shape (1,)